In [1]:
import numpy as np
import pandas as pd
from numpy.random import normal
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from RMSProp import SGD_RMSProp
from BetaRegression import *

# Проверка SGD RMSProp на данных в модели бета-регрессии 

In [2]:
df = pd.read_csv("./Data/observations.csv", sep=";").drop("Cover_class", axis = 1)
df.head()

,Site_id,Visit_id,Species_id,Cover
0,4,6345,2338,2
1,4,16199,2338,3
2,4,28382,2338,3
3,4,40703,2338,2
4,4,49283,2338,1


In [3]:
X, y = df.drop("Cover", axis=1), df["Cover"] / 100
nrow, ncol = X.shape

In [4]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [23]:
%%time

np.random.seed(42)

beta_start = normal(0, 1, ncol) ** 2
phi_start = normal(0, 1, 1) ** 2
start_point = np.append(beta_start, phi_start)

beta_res = SGD_RMSProp(
    start=start_point,
    X=X_train_s,
    y=y.to_numpy(),
    L_grad=beta_illh_grad,
    L=beta_inv_log_likelyhood,
    batch_size=100,
    learning_rate=0.01,
    decay_rate=0.5,
    max_iter=1000,
    tol=1e-3,
    link_inverse=logit_inverse,
    link_deriv=logit_deriv,
)

CPU times: user 137 ms, sys: 1.02 ms, total: 138 ms
Wall time: 137 ms


In [24]:
beta_res

{'point': array([0.01426504, 0.036983  , 0.03172449, 0.85047022]),
 'L_value': np.float64(-0.36802307402309),
 'grad_value': array([ 0.05925255, -0.0411079 , -0.02431094, -0.03777294]),
 'iterations': 152}